In [2]:
# importing libs
import seaborn as sns
import tqdm
from tqdm import tqdm
import sklearn
from sklearn.metrics import *
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [3]:
val_files = os.listdir('val/')
id = []
result = []

In [5]:
for i in range(0,len(val_files),1):
    a = val_files[i].replace('.jpeg','')
    id.append(a)
    if a.find('NORMAL'):
        result.append(1)
    else:
        result.append(0)

In [6]:
normal_xray = len(os.listdir('train/NORMAL'))
sick_xray = len(os.listdir('train/PNEUMONIA'))
number_train = normal_xray + sick_xray

In [7]:
normal_xray_test = len(os.listdir('test/NORMAL'))
sick_xray_test = len(os.listdir('test/PNEUMONIA'))
number_test = normal_xray_test + sick_xray_test

In [8]:
# building CNN with Keras
clf = Sequential()
clf.add(Conv2D(64, (3,3), input_shape = (64, 64, 3), activation = 'relu'))
clf.add(BatchNormalization())
clf.add(MaxPooling2D(pool_size = (2,2)))

clf.add(Conv2D(64, (3,3), input_shape = (64, 64, 3), activation = 'sigmoid'))
clf.add(BatchNormalization())
clf.add(MaxPooling2D(pool_size = (2,2)))

clf.add(Conv2D(64, (3,3), input_shape = (64, 64, 3), activation = 'relu'))
clf.add(BatchNormalization())
clf.add(MaxPooling2D(pool_size = (2,2)))

clf.add(Flatten())

clf.add(Dense(units = 128, activation = 'relu'))
clf.add(Dropout(0.2))
clf.add(Dense(units = 128, activation = 'relu'))
clf.add(Dropout(0.2))
clf.add(Dense(units = 128, activation = 'relu'))
clf.add(Dropout(0.2))
clf.add(Dense(units = 128, activation = 'relu'))
clf.add(Dropout(0.2))
clf.add(Dense(units = 128, activation = 'relu'))
clf.add(Dropout(0.2))
clf.add(Dense(units = 128, activation = 'relu'))
clf.add(Dropout(0.2))
clf.add(Dense(units = 128, activation = 'relu'))
clf.add(Dropout(0.2))
clf.add(Dense(units = 128, activation = 'relu'))
clf.add(Dropout(0.2))
clf.add(Dense(units = 128, activation = 'relu'))
clf.add(Dropout(0.2))
clf.add(Dense(units = 128, activation = 'relu'))
clf.add(Dropout(0.2))
clf.add(Dense(units = 128, activation = 'relu'))
clf.add(Dropout(0.2))
clf.add(Dense(units = 1, activation = 'sigmoid'))

clf.compile(optimizer = 'adam', loss = 'binary_crossentropy',
                      metrics = ['accuracy'])

In [9]:
generator_train = ImageDataGenerator(rescale = 1./255,
                                         rotation_range = 7,
                                         horizontal_flip = True,
                                         shear_range = 0.2,
                                         height_shift_range = 0.07,
                                         zoom_range = 0.2,
                                         fill_mode='nearest')

generator_test = ImageDataGenerator(rescale = 1./255)

base_train = generator_train.flow_from_directory('train',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

base_test = generator_test.flow_from_directory('test',
                                               target_size = (64, 64),
                                               batch_size = 32,
                                               class_mode = 'binary')

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [10]:
model = clf.fit_generator(base_train, steps_per_epoch = number_train/32,
                            epochs = 100, validation_data = base_test,
                            validation_steps = number_test, callbacks=False)

Epoch 1/50
163/163 [==============================] - 155s 942ms/step - loss: 0.4063 - accuracy: 0.7884 - val_loss: 0.5688 - val_accuracy: 0.6378
Epoch 2/50
163/163 [==============================] - 124s 759ms/step - loss: 0.2559 - accuracy: 0.9051
Epoch 3/50
163/163 [==============================] - 112s 684ms/step - loss: 0.2172 - accuracy: 0.9174
Epoch 4/50
163/163 [==============================] - 109s 668ms/step - loss: 0.2353 - accuracy: 0.9062
Epoch 5/50
163/163 [==============================] - 103s 629ms/step - loss: 0.2040 - accuracy: 0.9286
Epoch 6/50
163/163 [==============================] - 98s 600ms/step - loss: 0.2112 - accuracy: 0.9307
Epoch 7/50
163/163 [==============================] - 103s 631ms/step - loss: 0.1550 - accuracy: 0.9469
Epoch 8/50
163/163 [==============================] - 119s 732ms/step - loss: 0.1721 - accuracy: 0.9377
Epoch 9/50
163/163 [==============================] - 111s 678ms/step - loss: 0.1621 - accuracy: 0.9380
Epoch 10/50
163/163 [==

KeyboardInterrupt: 

In [ ]:
# showing the cnn structure
print(clf.summary())

In [ ]:
# summarize history for accuracy
plt.plot(model.history['accuracy'])
plt.title('Model Accuracy' , size =16 , weight = 'bold')
plt.ylabel('Accuracy' , size = 13 , weight = 'bold')
plt.xlabel('Epoch' , size =13 , weight = 'bold')
plt.grid()
plt.show()

In [ ]:
# summarize history for loss
plt.plot(model.history['loss'] , color='red')
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.grid()
plt.show()

In [ ]:
# normal xray example
img = mpimg.imread('val/'+val_files[0])
imgplot = plt.imshow(img)
plt.title('X-Ray of Normal Lung' , size = 15 , weight = 'bold')
plt.show()

In [ ]:
# sick xray example
img = mpimg.imread('val/'+val_files[11])
imgplot = plt.imshow(img)
plt.title('X-Ray of Sick Lung (Pneumonia)' , size = 15 , weight = 'bold')
plt.show()

In [ ]:
final_predict = []

tqdm.pandas()
for i in tqdm(range(0, len(val_files), 1)):
    img_test = image.load_img('val/'+val_files[i] ,
                                  target_size = (64,64))
    img_test = image.img_to_array(img_test)
    img_test /= 255
    img_test = np.expand_dims(img_test, axis = 0)
    previsao = clf.predict(img_test)
    final_predict.append(previsao[0][0])

In [ ]:
df = pd.DataFrame({'id':id , 'label':result , 'predict':final_predict})

In [ ]:
print(base_train.class_indices)

In [ ]:
df['predict'] = df['predict'].apply(lambda x: 1 if x > 0.5 else 0)

In [ ]:
acc = accuracy_score(df['label'] ,df['predict']) 
print('CNN Accuracy: ',acc*100, '%')

In [ ]:
# confusion matrix
cm = confusion_matrix(df['label'] ,df['predict'] )
sns.heatmap(cm, annot=True , cmap ='magma')
plt.title('Confusion Matrix' , size = 15 , weight = 'bold')
plt.show()